# A guide Portfolio Optimization Environment

This notebook aims to provide an example of using PortfolioOptimizationEnv (or POE) to train a reinforcement learning model that learns to solve the portfolio optimization problem.

In this document, we will reproduce a famous architecture called EIIE (ensemble of identical independent evaluators), introduced in the following paper:

- Zhengyao Jiang, Dixing Xu, & Jinjun Liang. (2017). A Deep Reinforcement Learning Framework for the Financial Portfolio Management Problem. https://doi.org/10.48550/arXiv.1706.10059.

It's advisable to read it to understand the algorithm implemented in this notebook.

### Note
If you're using this environment, consider citing the following paper (in adittion to FinRL references):

- Caio Costa, & Anna Costa (2023). POE: A General Portfolio Optimization Environment for FinRL. In *Anais do II Brazilian Workshop on Artificial Intelligence in Finance* (pp. 132–143). SBC. https://doi.org/10.5753/bwaif.2023.231144.

```
@inproceedings{bwaif,
 author = {Caio Costa and Anna Costa},
 title = {POE: A General Portfolio Optimization Environment for FinRL},
 booktitle = {Anais do II Brazilian Workshop on Artificial Intelligence in Finance},
 location = {João Pessoa/PB},
 year = {2023},
 keywords = {},
 issn = {0000-0000},
 pages = {132--143},
 publisher = {SBC},
 address = {Porto Alegre, RS, Brasil},
 doi = {10.5753/bwaif.2023.231144},
 url = {https://sol.sbc.org.br/index.php/bwaif/article/view/24959}
}

```

## Installation and imports

To run this notebook in google colab, uncomment the cells below.

In [1]:
## install finrl library
# !pip install wrds
# !pip install quantstats
# !pip install torch_geometric
# !pip install swig
# !pip install -q condacolab
# !pip install shimmy
# import condacolab
# condacolab.install()
# !apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
# !pip install git+https://github.com/flpymonkey/FinRL_Online_Portfolio_Benchmarks.git

In [2]:
## Hide matplotlib warnings
# import warnings
# warnings.filterwarnings('ignore')

import logging
logging.getLogger('matplotlib.font_manager').disabled = True

#### Import the necessary code libraries

In [3]:
import torch

import numpy as np

from sklearn.preprocessing import MaxAbsScaler

from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import GroupByScaler
from finrl.meta.env_portfolio_optimization.env_portfolio_optimization import PortfolioOptimizationEnv
from finrl.agents.portfolio_optimization.models import DRLAgent
from finrl.agents.portfolio_optimization.architectures import EIIE

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## Fetch data

In his paper, *Jiang et al* creates a portfolio composed by the top-11 cryptocurrencies based on 30-days volume. Since it's not specified when this classification was done, it's difficult to reproduce, so we will use a similar approach in the Brazillian stock market:

- We select top-10 stocks from Brazillian stock market;
- For simplicity, we disconsider stocks that have missing data for the days in period 2011-01-01 to 2019-12-31 (9 years);

In [4]:
TEST_TICKER = [
   "MSFT",
    "V",
    "AAPL",
    "BA",
    "INTC",
    "WMT",
]


TRAIN_START_DATE = '2009-04-01'
TEST_END_DATE = '2024-10-01'

TRAIN_DATES = ['2018-12-31']


In [5]:
print(len(TEST_TICKER))

portfolio_raw_df = YahooDownloader(start_date = TRAIN_START_DATE,
                                end_date = TEST_END_DATE,
                                ticker_list = TEST_TICKER).fetch_data()
portfolio_raw_df

6


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Shape of DataFrame:  (23406, 8)


,date,open,high,low,close,volume,tic,day
0,2009-04-01,3.717500,3.892857,3.710357,3.274470,589372000,AAPL,2
1,2009-04-01,34.520000,35.599998,34.209999,26.850760,9288800,BA,2
2,2009-04-01,14.770000,15.320000,14.620000,9.507452,75052800,INTC,2
3,2009-04-01,18.230000,19.360001,18.180000,14.301789,96438900,MSFT,2
4,2009-04-01,13.687500,14.000000,13.407500,12.129696,44144400,V,2
...,...,...,...,...,...,...,...,...
23401,2024-09-30,154.789993,155.300003,151.240005,152.039993,10902200,BA,0
23402,2024-09-30,23.740000,23.950001,23.090000,23.459999,66308200,INTC,0
23403,2024-09-30,428.209991,430.420013,425.369995,429.440399,16807300,MSFT,0
23404,2024-09-30,275.000000,275.690002,273.200012,274.428284,5969900,V,0


In [6]:
portfolio_raw_df.groupby("tic").count()

,date,open,high,low,close,volume,day
tic,,,,,,,
AAPL,3901,3901,3901,3901,3901,3901,3901
BA,3901,3901,3901,3901,3901,3901,3901
INTC,3901,3901,3901,3901,3901,3901,3901
MSFT,3901,3901,3901,3901,3901,3901,3901
V,3901,3901,3901,3901,3901,3901,3901
WMT,3901,3901,3901,3901,3901,3901,3901


In [7]:
INDICATORS = ['macd',
            'rsi_30',
            'cci_30',
            'dx_30']

from finrl.meta.preprocessor.preprocessors import FeatureEngineer

fe = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence=True,
                     user_defined_feature = False)

processed = fe.preprocess_data(portfolio_raw_df)
processed = processed.copy()
processed = processed.fillna(0)
processed = processed.replace(np.inf,0)

# # here you can see the engineered features are added to each stock day
print(processed)

Successfully added technical indicators
Successfully added turbulence index
             date        open        high         low       close     volume  \
0      2009-04-01    3.717500    3.892857    3.710357    3.274470  589372000   
1      2009-04-01   34.520000   35.599998   34.209999   26.850760    9288800   
2      2009-04-01   14.770000   15.320000   14.620000    9.507452   75052800   
3      2009-04-01   18.230000   19.360001   18.180000   14.301789   96438900   
4      2009-04-01   13.687500   14.000000   13.407500   12.129696   44144400   
...           ...         ...         ...         ...         ...        ...   
23401  2024-09-30  154.789993  155.300003  151.240005  152.039993   10902200   
23402  2024-09-30   23.740000   23.950001   23.090000   23.459999   66308200   
23403  2024-09-30  428.209991  430.420013  425.369995  429.440399   16807300   
23404  2024-09-30  275.000000  275.690002  273.200012  274.428284    5969900   
23405  2024-09-30   79.889999   80.940002   

### Normalize Data

We normalize the data dividing the time series of each stock by its maximum value, so that the dataframe contains values between 0 and 1.

In [8]:
portfolio_norm_df = GroupByScaler(by="tic", scaler=MaxAbsScaler).fit_transform(processed)
portfolio_norm_df

c:\Users\bencj\Desktop\Econ4\Thesis\.venv\lib\site-packages\finrl\meta\preprocessor\preprocessors.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.31332941 0.43188074 0.33824661 ... 0.01947727 0.01808933 0.02899626]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(
c:\Users\bencj\Desktop\Econ4\Thesis\.venv\lib\site-packages\finrl\meta\preprocessor\preprocessors.py:101: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.5  0.75 1.   ... 0.75 1.   0.  ]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  X.loc[select_mask, self.columns] = self.scalers[value].transform(


,date,open,high,low,close,volume,tic,day,macd,rsi_30,cci_30,dx_30,turbulence
0,2009-04-01,0.015720,0.016410,0.015918,0.013976,0.313329,AAPL,0.5,0.000000,1.000000,0.168245,1.000000,0.000000
1,2009-04-01,0.077397,0.079819,0.077716,0.062400,0.089997,BA,0.5,0.000000,1.000000,0.150477,1.000000,0.000000
2,2009-04-01,0.216569,0.221100,0.217204,0.153140,0.249431,INTC,0.5,0.000000,1.000000,0.120880,1.000000,0.000000
3,2009-04-01,0.039036,0.041337,0.039142,0.030705,0.302015,MSFT,0.5,0.000000,1.000000,0.148421,1.000000,0.000000
4,2009-04-01,0.046901,0.047770,0.046220,0.041682,0.130785,V,0.5,0.000000,1.000000,0.164371,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23401,2024-09-30,0.347055,0.348198,0.343579,0.353335,0.105628,BA,0.0,-0.075790,0.406534,-0.218247,0.383607,0.046359
23402,2024-09-30,0.348094,0.345649,0.343040,0.377879,0.220369,INTC,0.0,0.123309,0.489032,0.254049,0.246875,0.046359
23403,2024-09-30,0.916938,0.919014,0.915838,0.921968,0.052635,MSFT,0.0,0.309170,0.525853,0.114053,0.057228,0.046359
23404,2024-09-30,0.942297,0.940697,0.941809,0.943031,0.017687,V,0.0,0.053418,0.505272,-0.075579,0.105993,0.046359


### Instantiate Environment

Using the `PortfolioOptimizationEnv`, it's easy to instantiate a portfolio optimization environment for reinforcement learning agents. In the example below, we use the dataframe created before to start an environment.

### Instantiate Model

Now, we can instantiate the model using FinRL API. In this example, we are going to use the EIIE architecture introduced by Jiang et. al.

:exclamation: **Note:** Remember to set the architecture's `time_window` parameter with the same value of the environment's `time_window`.

In [9]:
df_portfolio = portfolio_norm_df[["date", "tic", "close", "high", "low", 'macd',
            'rsi_30',
            'cci_30',
            'dx_30']]

from datetime import datetime

NUM_MODELS = 5


for split_date in TRAIN_DATES:
    df_portfolio_train = df_portfolio[(df_portfolio["date"] >= TRAIN_START_DATE) & (df_portfolio["date"] < split_date)]

    df_portfolio_test = df_portfolio[(df_portfolio["date"] > split_date) & (df_portfolio["date"] < TEST_END_DATE)]

    for i in range(0, NUM_MODELS):

        print("=================")
        print(split_date)
        print(i)

        environment = PortfolioOptimizationEnv(
                df_portfolio_train,
                initial_amount=100000,
                comission_fee_pct=0.0025,
                time_window=50,
                features=["close", "high", "low", 'macd',
                    'rsi_30',
                    'cci_30',
                    'dx_30'],
                normalize_df=None
            )

        # set PolicyGradient parameters
        model_kwargs = {
            "lr": 0.01,
            "policy": EIIE,
        }

        # here, we can set EIIE's parameters
        policy_kwargs = {
            "k_size": 3,
            "initial_features":7,
            "time_window": 50,
        }

        model = DRLAgent(environment).get_model("pg", device, model_kwargs, policy_kwargs)

        DRLAgent.train_model(model, episodes=40)

        current_timestamp = datetime.now()
        timestamp_string = current_timestamp.strftime("%Y-%m-%d %H:%M:%S")

        # FILE = f"policy_EIIE_{split_date}_{str(i)}_{timestamp_string}.pt"

        torch.save(model.train_policy.state_dict(), "policy_EIIE.pt")

        environment_test = PortfolioOptimizationEnv(
            df_portfolio_test,
            initial_amount=100000,
            comission_fee_pct=0.0025,
            time_window=50,
            features=["close", "high", "low", 'macd',
            'rsi_30',
            'cci_30',
            'dx_30'],
            normalize_df=None
        )

        EIIE_results = {
            "training": environment._asset_memory["final"],
            "test": {},
        }

        # instantiate an architecture with the same arguments used in training
        # and load with load_state_dict.
        policy = EIIE(time_window=50, device=device, initial_features=7, k_size=3)
        # policy.load_state_dict(torch.load(FILE))
        
        policy.load_state_dict(torch.load("policy_EIIE.pt"))

        environment.reset()
        DRLAgent.DRL_validation(model, environment, policy=policy)
        EIIE_results["training"] = environment._asset_memory["final"]


        # 2020
        DRLAgent.DRL_validation(model, environment_test, policy=policy)
        EIIE_results["test"]["value"] = environment_test._asset_memory["final"]

        UBAH_results = {
            "train": {},
            "test": {},
        }

        PORTFOLIO_SIZE = len(TEST_TICKER)

        # train period
        terminated = False
        environment.reset()
        while not terminated:
            action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
            _, _, terminated, _ = environment.step(action)
        UBAH_results["train"]["value"] = environment._asset_memory["final"]

        # 2020
        terminated = False
        environment_test.reset()
        while not terminated:
            action = [0] + [1/PORTFOLIO_SIZE] * PORTFOLIO_SIZE
            _, _, terminated, _ = environment_test.step(action)
        UBAH_results["test"]["value"] = environment_test._asset_memory["final"]


2018-12-31
0


  0%|          | 0/40 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 510441.3125
Final accumulative portfolio value: 5.104413125
Maximum DrawDown: -0.2229776370713329
Sharpe ratio: 1.2102972323360868
Total commission cost: 16618.113236064644


  2%|▎         | 1/40 [00:22<14:52, 22.89s/it]

Initial portfolio value:100000
Final portfolio value: 585990.3125
Final accumulative portfolio value: 5.859903125
Maximum DrawDown: -0.22656560232214218
Sharpe ratio: 1.1775749641858582
Total commission cost: 62283.343335080746


...

Initial portfolio value:100000
Final portfolio value: 1214324.25
Final accumulative portfolio value: 12.1432425
Maximum DrawDown: -0.23330667193947763
Sharpe ratio: 1.3004807963935843
Total commission cost: 13702555.445240425


 48%|████▊     | 19/40 [08:15<09:26, 26.96s/it]

Initial portfolio value:100000
Final portfolio value: 1400853.375
Final accumulative portfolio value: 14.00853375
Maximum DrawDown: -0.22900821468709975
Sharpe ratio: 1.3648268711529572
Total commission cost: 14896741.25994077


 50%|█████     | 20/40 [08:54<08:54, 26.74s/it]


KeyboardInterrupt: 

### Train Model

  0%|          | 0/40 [00:00<?, ?it/s]

Initial portfolio value:100000
Final portfolio value: 1958396.5
Final accumulative portfolio value: 19.583965
Maximum DrawDown: -0.30520120295900544
Sharpe ratio: 1.1565652909453532


  2%|▎         | 1/40 [00:37<24:26, 37.59s/it]

Initial portfolio value:100000
Final portfolio value: 2673373.5
Final accumulative portfolio value: 26.733735
Maximum DrawDown: -0.3001341818144375
Sharpe ratio: 1.1769115049119097


  5%|▌         | 2/40 [01:23<27:03, 42.72s/it]

...

 95%|█████████▌| 38/40 [26:16<01:26, 43.18s/it]

Initial portfolio value:100000
Final portfolio value: 4087476.5
Final accumulative portfolio value: 40.874765
Maximum DrawDown: -0.3565325258093377
Sharpe ratio: 1.3730398519367388


 98%|█████████▊| 39/40 [26:54<00:41, 41.61s/it]

Initial portfolio value:100000
Final portfolio value: 3881257.5
Final accumulative portfolio value: 38.812575
Maximum DrawDown: -0.35653697422205255
Sharpe ratio: 1.3609118883892033


100%|██████████| 40/40 [27:29<00:00, 41.24s/it]


### Save Model

## Test Model

### Instantiate different environments

Since we have three different periods of time, we need three different environments instantiated to simulate them.

### Test EIIE architecture
Now, we can test the EIIE architecture in the three different test periods. It's important no note that, in this code, we load the saved policy even though it's not necessary just to show how to save and load your model.

### Test Uniform Buy and Hold
For comparison, we will also test the performance of a uniform buy and hold strategy. In this strategy, the portfolio has no remaining cash and the same percentage of money is allocated in each asset.

### Plot graphics

We can see that the agent is able to learn a good policy but its performance is worse the more the test period advances into the future. To get a better performance in 2022, for example, the agent should probably be trained again using more recent data.